# Restricted Boltzmann Machine for Large Dataset (ml-latest)

In [2]:
import csv
import numpy as np
from sklearn.neural_network import BernoulliRBM

## Loading Data

In [3]:
movies = []

with open('ml-latest/movies.csv', encoding='utf-8') as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        if i != 0:
            m_id = int(row[0])
            movies.append(m_id)

# retrieves movie index based on movie id
m_index_lookup = {}
            
for i, m_id in enumerate(movies):
    m_index_lookup[m_id] = i
    
n_movies = len(movies)
print("# of movies:", n_movies)

# of movies: 58098


This dataset is too large to store in memory, so we need to select a portion of the users.

### Ranking users based on how many movies rated

In [4]:
users = {}

with open('ml-latest/ratings.csv', encoding='utf-8') as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        if i != 0:
            u_id = int(row[0])
            if u_id not in users:
                users[u_id] = 1
            else:
                users[u_id] += 1
            
rankings = list(users.items())
rankings.sort(key = lambda x: x[1], reverse = True)
for i in range(10):
    print(rankings[i])

(123100, 23715)
(117490, 9279)
(134596, 8381)
(212343, 7884)
(242683, 7515)
(111908, 6645)
(77609, 6398)
(63783, 6346)
(172357, 5868)
(141955, 5810)


**Lets choose the top 5000 users**

In [5]:
top_users = set(rankings[i][0] for i in range(5000))
print(len(top_users))

5000


### Reading User Ratings

In [6]:
# 3.0 is threshold for favorable rating
def convert_rating(rating: str):
    r = float(rating)
    if (r < 3.0):
        return 0
    return 1

users = np.full((5000, n_movies), -1, dtype=np.int8)

curr_id = 0
index = 0

with open('ml-latest/ratings.csv', encoding='utf-8') as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        if i == 0:
            continue
        
        u_id = int(row[0])
        if u_id not in top_users:
            continue
            
        if curr_id == 0:
            curr_id = u_id
        elif u_id != curr_id:
            index += 1
            curr_id = u_id
        m_id = int(row[1])
        m_index = m_index_lookup[m_id]
        m_rating = convert_rating(row[2])
        users[index][m_index] = m_rating

print("users shape:", users.shape)

users shape: (5000, 58098)


In [7]:
def average_ratings_per_user(users):
    total = 0
    for user in users:
        # element-wise addition
        arr = user + 1
        # numpy has an advanced feature called Boolean Array Indexing
        # sets each element in arr that is greater than 0 to 1
        arr[arr > 0] = 1
        total += np.sum(arr)
    print("average # ratings per user:", total / len(users))
    
average_ratings_per_user(users)

average # ratings per user: 1237.9444


## Processing Data

In [8]:
def sort_rows(arr, col=0):
    return arr[np.argsort(arr[:, col])]

# selects k rows with the highest # of ratings
# TODO: update movie index
def filter(arr, k):
    arr2 = np.copy(arr)
    arr2 += 1
    arr2[arr2 > 0] = 1
    sums = np.empty((arr2.shape[0], 2))
    for i, row in enumerate(arr2):
        sums[i, 0] = np.sum(row)
        sums[i, 1] = i
    sums = sort_rows(sums)
    sums = sums[::-1] # reverse array
#     for i in range(k):
#         print(sums[i])
    indices = sums[:k, 1].T.astype(int)
    indices = np.sort(indices)
#     print(indices)
    top_k = arr[indices]
#     print(top_k.shape)
    return top_k

In [9]:
users2 = filter(users.T, 5000).T
print(users2.shape)
average_ratings_per_user(users2)

(5000, 5000)
average # ratings per user: 1030.5642


## Splitting Data

In [11]:
IMPUTE_MODE = "default"

In [10]:
# Some users are tough reviewers; others are more forgiving
# This function will randomly assign 1s and 0s based on their current review probabilities
def impute_missing(X, mode="default"):
    if mode == "default":
        for i, row in enumerate(X):
            # calculate how likely the user will give
            # a posive review
            neg = np.sum(row == 0)
            pos = np.sum(row == 1)
            p = pos / (pos + neg)

            missing = row == -1
            imputed = np.random.rand(np.sum(missing))
            imputed = imputed < p
            row[missing] = imputed.astype(int)
            
    elif mode == "random":
        for row in X:
            missing = row == -1
            imputed = np.random.rand(np.sum(missing))
            imputed = imputed < 0.5
            row[missing] = imputed.astype(int)
            
    elif mode == "zero":
        X[X < 0] = 0
        
def split_data(data, mode="default"):
    pi = np.random.permutation(data.shape[0])
    split = int(data.shape[0] * 0.8)

    Xtr_missing = data[pi[:split], :]
    Xte_missing = data[pi[split:], :]

    Xtr, Xte = np.copy(Xtr_missing), np.copy(Xte_missing)

    # Imputing Missing Values
    impute_missing(Xtr, mode)
    impute_missing(Xte, mode)
    
    return Xtr, Xte, Xtr_missing, Xte_missing


In [12]:
Xtr, Xte, Xtr_missing, Xte_missing = split_data(users2, mode=IMPUTE_MODE)

print("training shape:", Xtr.shape)
print("testing shape:", Xte.shape)

training shape: (4000, 5000)
testing shape: (1000, 5000)


In [14]:
# free up memory!
users = None

## RBM Time!

In [13]:
rbm = BernoulliRBM(
    n_components = 100,
    learning_rate = 0.1,
    batch_size = 10,
    n_iter = 100,
    verbose = 0,
    random_state = 0
)

# this might take an hour or 2
rbm.fit(Xtr)

BernoulliRBM(batch_size=10, learning_rate=0.1, n_components=100, n_iter=100,
             random_state=0, verbose=0)

In [16]:
# is this good or bad?
print(rbm.score_samples(Xte).mean())

-3074.7298431236204


## Testing Model

In [20]:
def conceal_ratings(data, portion=0.3):
    # create mask marking all known ratings
    mask = data + 1
    mask[mask > 0] = 1

    concealed = np.copy(data)
    for user, mask_row in zip(concealed, mask):
        indices = mask_row.nonzero()[0]
        n_ratings = indices.shape[0]
        indices = np.random.permutation(indices)
        split = int(n_ratings * portion)
        # set 30% of known ratings to missing
        user[indices[:split]] = -1
        # turn off bits for the latter 70% part of the mask
        mask_row[indices[split:]] = 0
        
    return concealed, mask

def sample_missing(rbm, Xobs, n_iters=10, mode="default"):
    Xhat = np.copy(Xobs)
    # impute missing values
    impute_missing(Xhat, mode)
    # print("preprocess done")
    for i in range(n_iters):
        Xhat = rbm.gibbs(Xhat).astype(int)
        Xhat[Xobs >= 0] = Xobs[Xobs >= 0]
    return Xhat

In [19]:
Xte_concealed, mask = conceal_ratings(Xte_missing, 0.3)

# Xte_concealed will now be same as Xte_missing 
    #   but with 30% of known ratings concealed.
    # mask will mark the locations of all ratings

(1000, 5000) (1000, 5000)


In [21]:
Xte_predict = sample_missing(rbm, Xte_concealed, n_iters = 100, mode = IMPUTE_MODE)

In [22]:
num_checks = np.sum(mask)
arr1 = Xte_missing * mask
arr2 = Xte_predict * mask
result = arr1 ^ arr2
num_err = np.sum(result)

print("Errors:", num_err)
print("Total Checks:", num_checks)
print("Error Rate:", num_err / num_checks)
print("RMSE:", np.sqrt(num_err/num_checks))

Errors: 105904
Total Checks: 308922
Error Rate: 0.34281792815014794
RMSE: 0.5855065568805766


In [23]:
arr1_b = arr1.astype(bool)
arr2_b = arr2.astype(bool)

# false positive:
fp_arr = np.invert(arr1_b) & arr2_b
# false negative:
fn_arr = arr1_b & np.invert(arr2_b)
# true positive:
tp_arr = arr1_b & arr2_b

fp = np.sum(fp_arr)
fn = np.sum(fn_arr)
tp = np.sum(tp_arr)
# since we're checking only a handful of indices
# true negative has to be calculated from the 3 other values
tn = num_checks - (fp + fn + tp)

print("True Positives: ", tp)
print("True Negatives: ", tn)
print("False Positives:", fp)
print("False Negatives:", fn)

accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)

print()
print("Accuracy: ", accuracy)
print("Precision:", precision)
print("Recall:   ", recall)

True Positives:  183327
True Negatives:  19691
False Positives: 53567
False Negatives: 52337

Accuracy:  0.6571820718498521
Precision: 0.7738777681156973
Recall:    0.7779168646887094
